# Regression Week 1: Simple Linear Regression

In this notebook we will use data on house sales in King County to predict house prices using simple (one input) linear regression. You will:
* Use numpy Array and functions to compute important summary statistics
* Write a function to compute the Simple Linear Regression weights using the closed form solution
* Write a function to make predictions of the output given the input feature
* Turn the regression around to predict the input given the output
* Compare two different models for predicting house prices


# import libraries

In [1]:
import pandas as pd 
#from sklearn.datasets import  irisdata
import matplotlib.pyplot as plt 
%matplotlib inline


# Load house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = pd.read_csv('kc_house_data.csv')
sales[:10]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
5,7237550310,20140512T000000,1225000.0,4,4.50,5420,101930,1.0,0,0,...,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
6,1321400060,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
7,2008000270,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711
8,2414600126,20150415T000000,229500.0,3,1.00,1780,7470,1.0,0,0,...,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113
9,3793500160,20150312T000000,323000.0,3,2.50,1890,6560,2.0,0,0,...,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570


In [6]:
print(sales.shape ,'\n',sales.head())


(21613, 21) 
            id             date     price  bedrooms  bathrooms  sqft_living  \
0  7129300520  20141013T000000  221900.0         3       1.00         1180   
1  6414100192  20141209T000000  538000.0         3       2.25         2570   
2  5631500400  20150225T000000  180000.0         2       1.00          770   
3  2487200875  20141209T000000  604000.0         4       3.00         1960   
4  1954400510  20150218T000000  510000.0         3       2.00         1680   

   sqft_lot  floors  waterfront  view     ...      grade  sqft_above  \
0      5650     1.0           0     0     ...          7        1180   
1      7242     2.0           0     0     ...          7        2170   
2     10000     1.0           0     0     ...          6         770   
3      5000     1.0           0     0     ...          7        1050   
4      8080     1.0           0     0     ...          8        1680   

   sqft_basement  yr_built  yr_renovated  zipcode      lat     long  \
0            

In [9]:
x= sales.drop(['price','id','date'], axis=1 )
y= sales['price']
print(x.shape ,x ,y,sep = '\n')

(21613, 18)
       bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  view  \
0             3       1.00         1180      5650     1.0           0     0   
1             3       2.25         2570      7242     2.0           0     0   
2             2       1.00          770     10000     1.0           0     0   
3             4       3.00         1960      5000     1.0           0     0   
4             3       2.00         1680      8080     1.0           0     0   
5             4       4.50         5420    101930     1.0           0     0   
6             3       2.25         1715      6819     2.0           0     0   
7             3       1.50         1060      9711     1.0           0     0   
8             3       1.00         1780      7470     1.0           0     0   
9             3       2.50         1890      6560     2.0           0     0   
10            3       2.50         3560      9796     1.0           0     0   
11            2       1.00         1160 

# Split data into training and testing

We use seed=0 so that everyone running this notebook gets the same results. 
In practice, you may set a random seed  

In [10]:
from sklearn.model_selection import train_test_split
x_train ,x_test,y_train,y_test = train_test_split (x,y ,test_size= 0.2 ,random_state = 40)
print("x_train_shape={} y_train_shape={}".format(x_train.shape ,y_train.shape) )
print("x_test_shape={} y_test_shape={}".format(x_test.shape ,y_test.shape))

x_train_shape=(17290, 18) y_train_shape=(17290,)
x_test_shape=(4323, 18) y_test_shape=(4323,)


# Useful built in summary functions

In order to make use of the closed form solution as well as take advantage of built in functions we will review some important ones. In particular:
* Computing the sum of an Array
* Computing the arithmetic average (mean) of an Array
* multiplying Arrays by constants
* multiplying Arrays by other Arrays

In [11]:
# Let's compute the mean of the House Prices in King County in 2 different ways.
prices = sales['price'] # extract the price column of the sales  

# recall that the arithmetic average (the mean) is the sum of the prices divided by the total number of houses:
sum_prices = prices.sum()
num_houses = prices.shape          # shape returns its length
avg_price_1 = sum_prices/num_houses
avg_price_2 = prices.mean() # if you just want the average, the .mean() function
print ('average price via method 1:' ,float(avg_price_1))
print ("average price via method 2: " ,float(avg_price_2))

average price via method 1: 540088.1417665294
average price via method 2:  540088.1417665294


As we see we get the same answer both ways

In [12]:
# if we want to multiply every price by 0.5 it's a simple as:
half_prices = 0.5*prices
half_prices

0         110950.0
1         269000.0
2          90000.0
3         302000.0
4         255000.0
5         612500.0
6         128750.0
7         145925.0
8         114750.0
9         161500.0
10        331250.0
11        234000.0
12        155000.0
13        200000.0
14        265000.0
15        325000.0
16        197500.0
17        242500.0
18         94500.0
19        115000.0
20        192500.0
21       1000000.0
22        142500.0
23        126350.0
24        164500.0
25        116500.0
26        468500.0
27        333500.0
28        219000.0
29        359500.0
           ...    
21583     199975.0
21584     190000.0
21585     135000.0
21586     252500.0
21587     192500.0
21588     207250.0
21589     173750.0
21590     611250.0
21591     286000.0
21592     237500.0
21593     544000.0
21594     175000.0
21595     260000.0
21596     339975.0
21597     787500.0
21598     270900.0
21599     405000.0
21600     768500.0
21601     233500.0
21602     112000.0
21603     253625.0
21604     21

In [14]:
# Let's compute the sum of squares of price. We can multiply two Arrays of the same length elementwise also with *
prices_squared = prices*prices
print('prices_squared',prices_squared)

prices_squared 0        4.923961e+10
1        2.894440e+11
2        3.240000e+10
3        3.648160e+11
4        2.601000e+11
5        1.500625e+12
6        6.630625e+10
7        8.517642e+10
8        5.267025e+10
9        1.043290e+11
10       4.389062e+11
11       2.190240e+11
12       9.610000e+10
13       1.600000e+11
14       2.809000e+11
15       4.225000e+11
16       1.560250e+11
17       2.352250e+11
18       3.572100e+10
19       5.290000e+10
20       1.482250e+11
21       4.000000e+12
22       8.122500e+10
23       6.385729e+10
24       1.082410e+11
25       5.428900e+10
26       8.779690e+11
27       4.448890e+11
28       1.918440e+11
29       5.169610e+11
             ...     
21583    1.599600e+11
21584    1.444000e+11
21585    7.290000e+10
21586    2.550250e+11
21587    1.482250e+11
21588    1.718102e+11
21589    1.207562e+11
21590    1.494506e+12
21591    3.271840e+11
21592    2.256250e+11
21593    1.183744e+12
21594    1.225000e+11
21595    2.704000e+11
21596    4.623320

In [15]:
sum_prices_squared = prices_squared.sum() # price_squared is an SArray of the squares and we want to add them up.
print ("the sum of price squared is: " ,float(sum_prices_squared))

the sum of price squared is:  9217325138472070.0


Aside: The python notation x.xxe+yy means x.xx \* 10^(yy). e.g 100 = 10^2 = 1*10^2 = 1e2 

# Build a generic simple linear regression function 

Armed with these Array functions we can use the closed form solution found from lecture to compute the slope and intercept for a simple linear regression on observations stored as Arrays: input_feature, output.

Complete the following function (or write your own) to compute the simple linear regression slope and intercept:

In [26]:
def simple_linear_regression(input_feature, output):
    # compute the sum of input_feature and output
    xi=input_feature
    yi=output
    N=len(xi)
    
    xmean=xi.mean()
    ymean=yi.mean()
    #print(xmean,ymean)
        
    # compute the product of the output and the input_feature and its sum
    
    sumxiyi=(xi*yi).sum()
    yixibyN=(yi.sum()*xi.sum())/N
    #print(sumxiyi , yixibyN)
    
    # compute the squared value of the input_feature and its sum
    
    xisq=(xi*xi).sum()
    
    xixibyN=(xi.sum()*xi.sum())/N
    #print(xisq , xixibyN)
 

    # use the formula for the slope

    slope=(sumxiyi-yixibyN)/(xisq-xixibyN)
    # use the formula for the intercept
    
    intercept=ymean-(slope*xmean)
    return (intercept, slope)

We can test that our function works by passing it something where we know the answer. In particular we can generate a feature and then put the output exactly on a line: output = 1 + 1\*input_feature then we know both our slope and intercept should be 1

In [17]:
import numpy as np
test_feature = np.array(range(5))
print(test_feature)

[0 1 2 3 4]


In [18]:
test_output = np.array(1 + 1*test_feature)
print(test_output)

[1 2 3 4 5]


In [27]:
# test the simple_linear_regression with test data
(test_intercept, test_slope) =  simple_linear_regression(test_feature, test_output)
print ("Intercept: " + str(test_intercept))
print ("Slope: " + str(test_slope))

Intercept: 1.0
Slope: 1.0


In [28]:
print(sales.columns)
print(x_train.columns) # after removal of some features such as id, date ...

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')
Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')


Now that we know it works let's build a regression model for predicting price based on sqft_living. Rembember that we train on train_data!

In [29]:
sqft_intercept, sqft_slope = simple_linear_regression(x_train['sqft_living'], y_train)

print ("Intercept: " + str(sqft_intercept))
print ("Slope: " + str(sqft_slope))

Intercept: -47399.245995822246
Slope: 282.4452087991784


# Predicting Values

Now that we have the model parameters: intercept & slope we can make predictions. Using Arrays it's easy to multiply an Array by a constant and add a constant value. Complete the following function to return the predicted output given the input_feature, slope and intercept:

In [30]:
def get_regression_predictions(input_feature, intercept, slope):
    # calculate the predicted values:
    # Y = MX + C is equation below 
    predicted_values=(slope*input_feature) + intercept
    
    return predicted_values

Now that we can calculate a prediction given the slope and intercept let's make a prediction. Use (or alter) the following to find out the estimated price for a house with 2650 squarefeet according to the squarefeet model we estiamted above.

**Quiz Question: Using your Slope and Intercept from above, What is the predicted price for a house with 2650 sqft?**

In [31]:
my_house_sqft = 2650
estimated_price = get_regression_predictions(my_house_sqft, sqft_intercept, sqft_slope)
print ("The estimated price for a house with {} sqaure feet is {}".format( my_house_sqft ,float( estimated_price)))

The estimated price for a house with 2650 sqaure feet is 701080.5573220005


# Residual Sum of Squares

Now that we have a model and can make predictions let's evaluate our model using Residual Sum of Squares (RSS). Recall that RSS is the sum of the squares of the residuals and the residuals is just a fancy word for the difference between the predicted output and the true output. 

Complete the following (or write your own) function to compute the RSS of a simple linear regression model given the input_feature, output, intercept and slope:

In [32]:
def get_residual_sum_of_squares(input_feature, output, intercept, slope):
    # First get the predictions
    
    predicted_values=intercept+(slope*input_feature)
    
    # then compute the residuals (since we are squaring it doesn't matter which order you subtract)
    residuals=output-predicted_values
    
    # square the residuals and add them up
    RSS=(residuals*residuals).sum()
    return(RSS)

Let's test our get_residual_sum_of_squares function by applying it to the test model where the data lie exactly on a line. Since they lie exactly on a line the residual sum of squares should be zero!

In [34]:
print(test_intercept, test_slope)

1.0 1.0


In [35]:
print (get_residual_sum_of_squares(x_test, y_test, test_intercept, test_slope)) 
# should be 0.0

bedrooms         0.0
bathrooms        0.0
sqft_living      0.0
sqft_lot         0.0
floors           0.0
waterfront       0.0
view             0.0
condition        0.0
grade            0.0
sqft_above       0.0
sqft_basement    0.0
yr_built         0.0
yr_renovated     0.0
zipcode          0.0
lat              0.0
long             0.0
sqft_living15    0.0
sqft_lot15       0.0
21568            0.0
3040             0.0
198              0.0
16456            0.0
19502            0.0
9988             0.0
6754             0.0
9725             0.0
9457             0.0
9688             0.0
9550             0.0
1556             0.0
                ... 
12233            0.0
9528             0.0
1536             0.0
20467            0.0
2424             0.0
12605            0.0
8759             0.0
9884             0.0
9762             0.0
19040            0.0
17041            0.0
14198            0.0
17428            0.0
20402            0.0
5703             0.0
5846             0.0
7209         

Now use your function to calculate the RSS on training data from the squarefeet model calculated above.

**Quiz Question: According to this function and the slope and intercept from the squarefeet model What is the RSS for the simple linear regression using squarefeet to predict prices on TRAINING data?**

In [36]:
rss_prices_on_sqft = get_residual_sum_of_squares(x_train['sqft_living'], y_train, sqft_intercept, sqft_slope)
print ('The RSS of predicting Prices based on Square Feet is :',float(rss_prices_on_sqft))

The RSS of predicting Prices based on Square Feet is : 1207358090826443.0


# Predict the squarefeet given price

What if we want to predict the squarefoot given the price? Since we have an equation y = a + b\*x we can solve the function for x. So that if we have the intercept (a) and the slope (b) and the price (y) we can solve for the estimated squarefeet (x).

Complete the following function to compute the inverse regression estimate, i.e. predict the input_feature given the output.

In [37]:
def inverse_regression_predictions(output, intercept, slope):
    # solve output = intercept + slope*input_feature for input_feature. Use this equation to compute the inverse predictions:
    estimated_feature=(output-intercept)/slope
    return estimated_feature

Now that we have a function to compute the squarefeet given the price from our simple regression model let's see how big we might expect a house that costs $800,000 to be.

**Quiz Question: According to this function and the regression slope and intercept from above. what is the estimated square-feet for a house costing $800,000?**

In [39]:
my_house_price = 800000
estimated_squarefeet = inverse_regression_predictions(my_house_price, sqft_intercept, sqft_slope)
print ("The estimated squarefeet for a house worth {} is {}".format(my_house_price,estimated_squarefeet))

The estimated squarefeet for a house worth 800000 is 3000.225245804514


# New Model: estimate prices from bedrooms

We have made one model for predicting house prices using squarefeet, but there are many other features in the sales dataset. 
Use your simple linear regression function to estimate the regression parameters from predicting Prices based on number of bedrooms. Use the training data!

In [40]:
# Estimate the slope and intercept for predicting 'price' based on 'bedrooms'
sqft_intercept, sqft_slope = simple_linear_regression(x_train['bedrooms'], y_train)

print ("Intercept: " + str(sqft_intercept))
print ("Slope: " + str(sqft_slope))


Intercept: 137172.4364897065
Slope: 119412.7650152319


# Test your Linear Regression Algorithm

Now we have two models for predicting the price of a house. How do we know which one is better? Calculate the RSS on the TEST data (remember this data wasn't involved in learning the model). Compute the RSS from predicting prices using bedrooms and from predicting prices using squarefeet.

**Quiz Question: Which model (square feet or bedrooms) has lowest RSS on TEST data? Think about why this might be the case.**

In [45]:
# Compute RSS when using bedrooms on TEST data:
sqft_intercept, sqft_slope = simple_linear_regression(x_train['bedrooms'],y_train )
rss_prices_on_bedrooms = get_residual_sum_of_squares(x_test['bedrooms'], y_test, sqft_intercept, sqft_slope)
print("The RSS of predicting Prices based on Bedrooms is",rss_prices_on_bedrooms)

The RSS of predicting Prices based on Bedrooms is 482832293053847.7


In [46]:
# Compute RSS when using squarefeet on TEST data:
sqft_intercept, sqft_slope = simple_linear_regression(x_train['sqft_living'], y_train)
rss_prices_on_sqft_living = get_residual_sum_of_squares(x_test['sqft_living'], y_test, sqft_intercept, sqft_slope)
print("The RSS of predicting Prices based on Bedrooms is",rss_prices_on_sqft_living)


The RSS of predicting Prices based on Bedrooms is 269978786005070.2


In [48]:
rss_prices_on_bedrooms<rss_prices_on_sqft_living
# bedrooms RSS is high

False

In [22]:
sales

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
5,7237550310,20140512T000000,1225000.0,4,4.50,5420,101930,1.0,0,0,...,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
6,1321400060,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
7,2008000270,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711
8,2414600126,20150415T000000,229500.0,3,1.00,1780,7470,1.0,0,0,...,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113
9,3793500160,20150312T000000,323000.0,3,2.50,1890,6560,2.0,0,0,...,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570
